CLASSIFICATION

Database from https://www.kaggle.com/competitions/titanic/data?select=train.csv

In [27]:
!pip install -q sklearn

In [28]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc


In [29]:
CSV_COLUMN_NAMES=['SepalLength','SepalWidth', 'PetalLength','PetalWidth','Species']
Species = ['Setosa','Versicolor','Virginica']

In [30]:
train_path=tf.keras.utils.get_file("iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path=tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train=pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test=pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)


In [31]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [32]:
train_y=train.pop('Species')
test_y=test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [33]:
def input_fn(features,labels,training=True,bach_size=256):
    dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
    if training:
        dataset=dataset.shuffle(1000).repeat()
    return dataset.batch(bach_size)


In [34]:
my_feature_columns=[]
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [35]:
# Build a DNN with 2 hiden layers with 30 and 10 nodes each on

classifier=tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # two hiden layers with 30 and 10 respectively
    hidden_units=[30,10],
    # Three classes to choose              
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp41qtmbzz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
classifier.train(input_fn=lambda: input_fn(train,train_y,training=True),
                                           steps=500)

#Include lambda to avoid creating an inner function previosly

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp41qtmbzz/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1154286, step = 0
INFO:tensorflow:global_step/sec: 466.803
INFO:tensorflow:loss = 0.93167555, step = 100 (0.215 sec)
INFO:tensorflow:global_step/sec: 529.761
INFO:tensorflow:loss = 0.88098294, step = 200 (0.189 sec)
INFO:tensorflow:global_step/sec: 592.33
INFO:tensorflow:loss = 0.82989925, step = 300 (0.169 sec)
INFO:tensorflow:global_step/sec: 517.824
INFO:tensorflow:loss = 0.7801691, step = 400 (0.194 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 500...
INFO:tensorflow:Saving ch

In [37]:
eval_result=classifier.evaluate(input_fn=lambda: input_fn(test,test_y,training=False))

print('\Test se accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-01-23T11:36:33
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp41qtmbzz/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.18921s
INFO:tensorflow:Finished evaluation at 2023-01-23-11:36:33
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.53333336, average_loss = 0.7992841, global_step = 500, loss = 0.7992841
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: /tmp/tmp41qtmbzz/model.ckpt-500
\Test se accuracy: 0.533



In [39]:
def  input_fn(features,batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict={}

print("Please type numeric values as prompted.")
for feature in features:
    valid= True
    while valid:
        val= input(feature + ": ")
        if not val.isdigit(): valid = False

    predict[feature]=[float(val)]
    
predictions=classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id= pred_dict['class_ids'][0]
    probability= pred_dict['probabilities'][class_id]
     
    print('Prediction is "{}" ({:1f}%)'.format(Species[class_id],100*probability))

Please type numeric values as prompted.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp41qtmbzz/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Versicolor" (87.724471%)
